In [1]:
!sudo apt-get update
!sudo apt-get install -y gcc perl

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,128 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,840 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [2]:
!wget https://raw.githubusercontent.com/torvalds/linux/master/scripts/checkpatch.pl
!chmod +x checkpatch.pl

--2025-07-19 19:54:50--  https://raw.githubusercontent.com/torvalds/linux/master/scripts/checkpatch.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 239423 (234K) [text/plain]
Saving to: ‘checkpatch.pl’

checkpatch.pl       100%[===================>] 233.81K  1.39MB/s    in 0.2s    

2025-07-19 19:54:51 (1.39 MB/s) - ‘checkpatch.pl’ saved [239423/239423]



In [3]:
import subprocess

def compile_driver(filepath):
    try:
        proc = subprocess.run(["gcc", "-Wall", "-Wextra", "-o", "temp.ko", filepath],
                              capture_output=True, text=True)
        warnings = proc.stderr.count("warning:")
        errors = proc.stderr.count("error:")

        return {
            "success": proc.returncode == 0,
            "warnings": warnings,
            "errors": errors
        }

    except Exception as e:
        return {"success": False, "error": str(e)}

compile_result = compile_driver("char_driver_sample.c")
compile_result

{'success': False, 'warnings': 0, 'errors': 1}

In [4]:
import subprocess

def compile_driver_debug(filepath):
    proc = subprocess.run(["gcc", "-Wall", "-Wextra", "-o", "temp.ko", filepath],
                          capture_output=True, text=True)

    print("STDOUT:\n", proc.stdout)
    print("STDERR:\n", proc.stderr)
    print("Return code:", proc.returncode)

    return {
        "success": proc.returncode == 0,
        "warnings": proc.stderr.count("warning:"),
        "errors": proc.stderr.count("error:")
    }

compile_driver_debug("char_driver_sample.c")

STDOUT:
 
STDERR:
 char_driver_sample.c:1:10: fatal error: linux/init.h: No such file or directory
    1 | #include <linux/init.h>
      |          ^~~~~~~~~~~~~~
compilation terminated.

Return code: 1


{'success': False, 'warnings': 0, 'errors': 1}

In [5]:
ls

char_driver_sample.c  checkpatch.pl*  sample_data/


In [6]:
compile_driver("char_driver_sample.c")

{'success': True, 'warnings': 0, 'errors': 0}

In [8]:
import subprocess

def check_style(filepath):
    try:
        result = subprocess.run(["perl", "checkpatch.pl", filepath],
                                capture_output=True, text=True)
        issues = result.stdout.count("WARNING:")
        errors = result.stdout.count("ERROR:")
        score = max(0, 1 - ((issues + errors) / 20))  # Scaled score out of 1

        return {
            "style_issues": issues,
            "style_errors": errors,
            "style_score": round(score, 2)
        }

    except Exception as e:
        return {"error": str(e)}


In [9]:
style_result = check_style("char_driver_sample.c")
style_result

{'style_issues': 0, 'style_errors': 0, 'style_score': 1.0}

In [10]:
def run_security_checks(filepath):
    try:
        with open(filepath, "r") as f:
            code = f.read()

        score = {
            "buffer_safety": 1.0 if "copy_from_user" in code or "strncpy" in code else 0.6,
            "race_conditions": 1.0 if ("mutex" in code or "spin_lock" in code) else 0.5,
            "input_validation": 1.0 if "if" in code and "return -EINVAL" in code else 0.4
        }

        return score

    except Exception as e:
        return {"error": str(e)}

In [11]:
security_result = run_security_checks("char_driver_sample.c")
security_result

{'buffer_safety': 1.0, 'race_conditions': 0.5, 'input_validation': 0.4}

In [12]:
def calculate_score(comp, style, security):
    weights = {
        "compilation": 0.4,
        "security": 0.25,
        "code_quality": 0.2,
        "performance": 0.1,  # Not used yet
        "advanced": 0.05     # Not used yet
    }

    score = 0

    if comp.get("success"):
        score += weights["compilation"] * 100

    score += weights["code_quality"] * style.get("style_score", 0) * 100
    sec_avg = sum(security.values()) / 3
    score += weights["security"] * sec_avg * 100

    return round(score, 2)

In [13]:
final_score = calculate_score(compile_result, style_result, security_result)
final_score

35.83

In [14]:
compile_result = {'success': True, 'warnings': 0, 'errors': 0}
style_result = {'style_issues': 0, 'style_errors': 0, 'style_score': 1.0}
security_result = {'buffer_safety': 1.0, 'race_conditions': 0.5, 'input_validation': 0.4}

In [15]:
compile_result = compile_driver("char_driver_sample.c")
style_result = check_style("char_driver_sample.c")
security_result = run_security_checks("char_driver_sample.c")
final_score = calculate_score(compile_result, style_result, security_result)
final_score

75.83

In [16]:
import json

evaluation_report = {
    "compilation": compile_result,
    "code_quality": style_result,
    "security": security_result,
    "overall_score": final_score
}

with open("evaluation_result.json", "w") as f:
    json.dump(evaluation_report, f, indent=4)

evaluation_report

{'compilation': {'success': True, 'warnings': 0, 'errors': 0},
 'code_quality': {'style_issues': 0, 'style_errors': 0, 'style_score': 1.0},
 'security': {'buffer_safety': 1.0,
  'race_conditions': 0.5,
  'input_validation': 0.4},
 'overall_score': 75.83}

In [17]:
from google.colab import files
files.download("evaluation_result.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
compile_result = compile_driver("llm_generated_driver.c")
style_result = check_style("llm_generated_driver.c")
security_result = run_security_checks("llm_generated_driver.c")
final_score = calculate_score(compile_result, style_result, security_result)

In [19]:
evaluation_report = {
    "compilation": compile_result,
    "code_quality": style_result,
    "security": security_result,
    "overall_score": final_score
}

In [20]:
with open("evaluation_llm.json", "w") as f:
    json.dump(evaluation_report, f, indent=4)

from google.colab import files
files.download("evaluation_llm.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
!gcc test_driver_sim.c -o sim_test
!./sim_test > results/runtime_test_output.txt

/bin/bash: line 1: results/runtime_test_output.txt: No such file or directory


In [23]:
!mkdir -p results

In [24]:
!gcc test_driver_sim.c -o sim_test
!./sim_test > results/runtime_test_output.txt

In [25]:
!cat results/runtime_test_output.txt

Written to buffer: Simulated user write!
Read from buffer: Simulated user write!
